<a href="https://colab.research.google.com/github/sergiokv13/03MAIR-Algoritmos-de-Optimizacion/blob/master/AG3/Algoritmos_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de Optimizacion - Actividad Guiada 3

<strong>Nombre:</strong> Sergio Andres Köller Vargas <br>
<strong>URL:</strong> https://colab.research.google.com/drive/1eHMpvkb9uojd5-0ITsIfQkHz_sgDMpdh?usp=sharing <br>
<strong>Github:</strong> https://github.com/sergiokv13/03MAIR-Algoritmos-de-Optimizacion/blob/master/AG3/Algoritmos_AG3.ipynb

In [ ]:
!pip install requests
!pip install tsplib95

In [ ]:
import tsplib95
import random
from math import e
import math
import copy

In [ ]:
import urllib.request
file = "swiss42.tsp"
urllib.request.urlretrieve(f"http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/{file}", file)

('swiss42.tsp', <http.client.HTTPMessage at 0x7f6f6c96d588>)

In [ ]:
# Carga de datos y generación de objeto problem

problem = tsplib95.load(file)

# Nodos
nodos = list(problem.get_nodes())

# Aristas
aristas = list(problem.get_edges())

In [ ]:
# Probamos algunas funciones

# Distancia entre nodos
problem.get_weight(0,1)

15

In [ ]:
# Formato de solución:

# El primer elemento siempre sera 0, seguido de las ciudades siguientes
# solucion = [0,1,2,3,...,41]

In [ ]:
# Funciones basicas

# Generar una solución aleatoria que comience en el nodo 0
def crear_solucion(nodos):
  solucion = [nodos[0]]
  for n in nodos[1:]:
    solucion = solucion + [random.choice(list(set(nodos) - set(solucion)))]
  return solucion

# Devuelve la distancia entre dos nodos
def distancia(a, b, problem):
  return problem.get_weight(a,b)

# Devuelve la distancia total de una trayectoria/solución
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion) - 1):
    distancia_total += distancia(solucion[i], solucion[i+1], problem)
    # Vuelva al nodo inicial
  return distancia_total + distancia(solucion[len(solucion)-1], solucion[0], problem)

### Busqueda Aleatoria

In [ ]:
def busqueda_aleatoria(problem, max_iter): 
  nodos = list(problem.get_nodes())
  mejor_solucion = []
  mejor_distancia = 10e100

  for _i in range(max_iter):
    solucion = crear_solucion(nodos)
    distancia = distancia_total(solucion, problem)

    if distancia < mejor_distancia:
      mejor_solucion = solucion
      mejor_distancia = distancia

  print("Mejor solución: ", mejor_solucion)
  print("Distancia     :", mejor_distancia)
  return mejor_solucion

sol = busqueda_aleatoria(problem, 10000)
    

Mejor solución:  [0, 12, 11, 32, 30, 1, 7, 35, 13, 14, 15, 4, 20, 38, 8, 23, 34, 37, 5, 36, 17, 33, 2, 10, 22, 6, 3, 21, 40, 29, 27, 18, 41, 25, 9, 39, 24, 16, 26, 28, 31, 19]
Distancia     : 3763


### Busqueda Local

In [ ]:
def genera_vecina(solucion):
  mejor_solucion = []
  mejor_distancia = 10e100

  for i in range(1,len(solucion)-1):
    for j in range(i+1, len(solucion)):
      # Generaremos vecinos intercambiando dos nodos i,j
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
      # Se evalua la nueva solución
      distancia_vecina = distancia_total(vecina, problem)

      # ... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina

  return mejor_solucion

solucion = [0, 4, 36, 17, 2, 40, 9, 12, 13, 19, 11, 8, 18, 25, 28, 30, 3, 35, 21, 39, 29, 33, 32, 26, 5, 16, 10, 38, 34, 24, 22, 41, 23, 7, 20, 14, 37, 15, 31, 1, 6, 27]
print("Distancia solución inicial: ", distancia_total(solucion, problem))
nueva_solucion = genera_vecina(solucion)
print("Distancia mejor solución vecina: ", distancia_total(nueva_solucion, problem))


Distancia solución inicial:  3537
Distancia mejor solución vecina:  3252


In [ ]:
# Algoritmo de busqueda local
def busqueda_local(problem, solucion_inicial = crear_solucion(nodos)): 
  mejor_solucion = solucion_inicial
  mejor_distancia = distancia_total(mejor_solucion, problem)

  iteracion = 0
  while(True):
    iteracion += 1
    vecina = genera_vecina(mejor_solucion)
    distancia_vecina = distancia_total(vecina, problem)

    if distancia_vecina < mejor_distancia:
      mejor_distancia = distancia_vecina
      mejor_solucion = copy.deepcopy(vecina)
    else:
      return mejor_solucion

sol = busqueda_local(problem)
print("Mejor solución: ", sol)
print("Distancia     :", distancia_total(sol, problem))


Mejor solución:  [0, 31, 20, 33, 34, 32, 28, 30, 38, 22, 39, 21, 9, 25, 11, 10, 8, 23, 41, 12, 18, 26, 37, 17, 35, 36, 15, 16, 14, 7, 1, 27, 2, 29, 24, 40, 13, 19, 5, 6, 4, 3]
Distancia     : 1889


### Recocido simulado

In [ ]:
# Generador de 1 solucion vecina aleatoria (intercambiar 2 nodos) 
def genera_vecina_aleatorio(solucion):
  # Se eligen dos nodos aleatoriamente
  i, j = sorted(random.sample(range(1, len(solucion)), 2))

  # Devuelve una nueva solución pero intercambiando los dos nodos elegidos al azar
  return solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

In [ ]:
# Funcion de probabilidad para aceptar peores soluciones
def probabilidad(t, d):
  return random.random() < math.exp(-1*d / t)

# Funcion de descenso de temperatura
def bajar_temperatura(t):
  return t*0.99

In [ ]:
def recocido_simulado(problem, temperatura, solucion_inicial = crear_solucion(nodos), genera_vecina_aleatorio=genera_vecina_aleatorio):
  solucion_referencia = solucion_inicial
  distancia_referencia = distancia_total(solucion_referencia, problem)

  mejor_solucion = []
  mejor_distancia = 10e100

  while(temperatura > .1):
    # Generar solucion vecina aleatoria
    vecina = genera_vecina_aleatorio(solucion_referencia)
    distancia_vecina = distancia_total(vecina, problem)

    # Si es la mejor solucion se guarda
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina

    # Si la nueva vecina es mejor, se cambia
    # Si es peor se cambia segun prob de t y delta (dist_ref - dist_vec)
    if distancia_vecina < distancia_referencia or probabilidad(temperatura, abs(distancia_referencia - distancia_vecina)):
      solucion_referencia = copy.deepcopy(vecina)
      distancia_referencia = distancia_vecina

    temperatura = bajar_temperatura(temperatura)
  return mejor_solucion

sol = recocido_simulado(problem, 10000)  

print("Mejor solución: ", sol)
print("Distancia     :", distancia_total(sol, problem))

Mejor solución:  [0, 6, 7, 1, 23, 21, 39, 40, 24, 22, 38, 3, 5, 14, 15, 16, 37, 20, 33, 36, 31, 17, 19, 25, 9, 8, 29, 32, 35, 34, 30, 27, 2, 13, 26, 18, 28, 10, 41, 11, 12, 4]
Distancia     : 2446


### Mejoras

** Notar que se editarón los metodos de busqueda local y de recocido simulado para recibir una función de vecindad y un nodo inicial.

#### Busqueda local con Entornos Variables

http://www.cs.us.es/~fsancho/?e=208

In [ ]:
# Generaremos el vecino a partir de una busqueda local normal (monótona)
# en caso de que esta nueva solucion no mejore la actual, entonces generaremos una nueva solucíon aleatoria
# para variar el entorno

# max_changes controla cuantos cambios de entorno aleatorios permitiremos

# VNS
def generar_vecina_mejorado_local(solucion, max_changes):
  nueva_solucion = busqueda_local(problem, solucion)
  if (distancia_total(nueva_solucion, problem) < distancia_total(solucion, problem)):
    return nueva_solucion, max_changes
  elif max_changes > 0: # Cambiamos de entorno buscando una mejor solución
    while(max_changes):
      nueva_solucion = busqueda_local(problem, crear_solucion(nodos))
      max_changes -= 1
      if (distancia_total(nueva_solucion, problem) < distancia_total(solucion, problem)):
        return nueva_solucion, max_changes
  return solucion, 0

In [ ]:
# Generamos un nuevo algoritmo de busqueda local mejorado
# max changes hara un control de cuantos cambios de entorno vamos a permitir para no quedarnos en un loop infinito
def busqueda_local_vns(problem, solucion_inicial = crear_solucion(nodos), max_changes = 100): 
  mejor_solucion = solucion_inicial
  mejor_distancia = distancia_total(mejor_solucion, problem)

  iteracion = 0
  while(True):
    iteracion += 1
    vecina, max_changes = generar_vecina_mejorado_local(mejor_solucion, max_changes)
    distancia_vecina = distancia_total(vecina, problem)

    if distancia_vecina < mejor_distancia:
      mejor_distancia = distancia_vecina
      mejor_solucion = copy.deepcopy(vecina)
    else:
      return mejor_solucion


In [ ]:
solucion_inicial = crear_solucion(nodos)
# Notar que estamos iniciando con la misma solucion inicial para que la 
# comparativa haga mas sentido
print("Con la busqueda local hecha en clase: ")
sol = busqueda_local(problem, solucion_inicial)
print("Mejor solución: ", sol)
print("Distancia     :", distancia_total(sol, problem))
print()
print("Con entornos variables:")
sol = busqueda_local_vns(problem, solucion_inicial, 100) 
print("Mejor solución: ", sol)
print("Distancia     :", distancia_total(sol, problem))

Con la busqueda local hecha en clase: 
Mejor solución:  [0, 18, 10, 29, 30, 32, 17, 37, 15, 16, 14, 24, 40, 23, 41, 25, 11, 12, 13, 19, 7, 34, 33, 20, 35, 36, 31, 38, 22, 39, 21, 9, 8, 28, 27, 2, 4, 26, 5, 6, 3, 1]
Distancia     : 1941

Con entornos variables:
Mejor solución:  [0, 1, 10, 41, 9, 8, 29, 38, 22, 39, 21, 24, 40, 23, 25, 11, 12, 18, 4, 3, 27, 2, 28, 30, 34, 33, 20, 32, 7, 31, 35, 36, 17, 37, 15, 16, 14, 19, 13, 5, 26, 6]
Distancia     : 1523


#### Recocido simulado con vecindad no tan aleatoria

In [ ]:
# Buscaremos los mejores 5 vecinos y escogeremos uno aleatorio
def generar_vecina_mejorado_sim_ann(solucion):
  soluciones = []

  for i in range(1,len(solucion)-1):
    for j in range(i+1, len(solucion)):
      # Generaremos vecinos intercambiando dos nodos i,j
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
      soluciones.append(vecina)

  soluciones.sort(key=lambda x: distancia_total(x, problem))
  return random.choice(soluciones[:5])

print(generar_vecina_mejorado(crear_solucion(nodos)))


[0, 7, 17, 6, 22, 3, 20, 35, 5, 28, 12, 37, 8, 41, 32, 39, 1, 34, 2, 24, 29, 36, 33, 19, 25, 9, 40, 18, 11, 16, 31, 38, 10, 27, 14, 4, 30, 23, 13, 15, 26, 21]


In [222]:
solucion_inicial = crear_solucion(nodos)
# Notar que estamos iniciando con la misma solucion inicial para que la 
# comparativa haga mas sentido
print("Con la elección aleatoria: ")
sol = recocido_simulado(problem, 10000, solucion_inicial) 
print("Mejor solución: ", sol)
print("Distancia     :", distancia_total(sol, problem))
print()
print("Con la elección mejorada: ")
sol = recocido_simulado(problem, 10000, solucion_inicial, generar_vecina_mejorado_sim_ann)
print("Mejor solución: ", sol)
print("Distancia     :", distancia_total(sol, problem))


Con la elección aleatoria: 
Mejor solución:  [0, 6, 2, 28, 27, 30, 21, 40, 24, 39, 29, 17, 31, 32, 4, 37, 14, 15, 16, 5, 26, 12, 9, 23, 41, 8, 1, 36, 35, 20, 33, 7, 19, 13, 3, 34, 38, 22, 10, 25, 11, 18]
Distancia     : 2335

Con la elección mejorada: 
Mejor solución:  [0, 26, 18, 12, 10, 9, 21, 40, 24, 39, 33, 35, 36, 17, 31, 32, 27, 2, 28, 29, 8, 23, 41, 25, 11, 13, 19, 14, 16, 15, 37, 20, 34, 38, 22, 30, 3, 4, 6, 5, 7, 1]
Distancia     : 1795
